In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import DBLPScholarStructuredBenchmark

benchmark = DBLPScholarStructuredBenchmark(data_dir_path="../data/")
benchmark

17:21:59 INFO:Extracting DBLP-Scholar-Structured...
17:21:59 INFO:Reading DBLP-Scholar-Structured row_dict...
17:21:59 INFO:Reading DBLP-Scholar-Structured train.csv...
17:22:00 INFO:Reading DBLP-Scholar-Structured valid.csv...
17:22:00 INFO:Reading DBLP-Scholar-Structured test.csv...


<DBLPScholarStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/DBLP-GoogleScholar/dblp_scholar_exp_data.zip

## Preprocess

In [5]:
field_list = ['title', 'authors', 'venue', 'year']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

  0%|          | 0/66879 [00:00<?, ?it/s]

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [10]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

17:22:00 INFO:For attr=title, computing actual max_str_len
17:22:00 INFO:For attr=title, using actual_max_str_len=70
17:22:01 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:22:04 INFO:For attr=authors, computing actual max_str_len
17:22:05 INFO:actual_max_str_len=33 must be even to enable NN pooling. Updating to 34
17:22:05 INFO:For attr=authors, using actual_max_str_len=34
17:22:05 INFO:For attr=venue, computing actual max_str_len
17:22:05 INFO:actual_max_str_len=45 must be even to enable NN pooling. Updating to 46
17:22:05 INFO:For attr=venue, using actual_max_str_len=46
17:22:05 INFO:Loading vectors from .vector_cache/wiki.en.vec.pt
17:22:09 INFO:For attr=year, computing actual max_str_len
17:22:09 INFO:For attr=year, using actual_max_str_len=6


{'title': AttrConfig(source_attr='title', field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=70, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'semantic_title': AttrConfig(source_attr='title', field_type=<FieldType.SEMANTIC: 'semantic_multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$'

In [11]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [12]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

In [13]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

GPU available: True, used: True
17:22:09 INFO:GPU available: True, used: True
TPU available: None, using: 0 TPU cores
17:22:09 INFO:TPU available: None, using: 0 TPU cores


In [ ]:
model.validate(datamodule)

## Testing

In [17]:
model.test(datamodule)

17:36:53 INFO:Train pair count: 3207
17:36:53 INFO:Valid pair count: 1070
17:36:53 INFO:Test pair count: 1070
17:36:53 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

[{'test_f1_at_0.3': 0.13798889892861751,
  'test_f1_at_0.5': 0.6645748352682774,
  'test_f1_at_0.7': 0.9400630914826499,
  'test_f1_at_0.9': 0.9125437281359321,
  'test_pair_entity_ratio_at_0.3': 7.5557883708748035,
  'test_pair_entity_ratio_at_0.5': 1.1089575694080671,
  'test_pair_entity_ratio_at_0.7': 0.6018858040859089,
  'test_pair_entity_ratio_at_0.9': 0.48768988999476165,
  'test_precision_at_0.3': 0.07411259012756517,
  'test_precision_at_0.5': 0.500236183278224,
  'test_precision_at_0.7': 0.9077458659704091,
  'test_precision_at_0.9': 0.9806659505907627,
  'test_recall_at_0.3': 0.9990654205607477,
  'test_recall_at_0.5': 0.9897196261682243,
  'test_recall_at_0.7': 0.974766355140187,
  'test_recall_at_0.9': 0.8532710280373832}]